In [ ]:
import torch
import numpy as np

from torch import optim, nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")
print(device)

In [ ]:
from tqdm import notebook
from scipy import stats
from scipy.stats import norm

In [ ]:
class my_dataset(Dataset):
    def __init__(self,data,label):
        self.data=x
        self.label=y          
    def __getitem__(self, index):
        return self.data[index],self.label[index]
    def __len__(self):
        return len(self.data)

### load test data

In [ ]:
xlr=np.load('xlr.npy')
test_x=torch.from_numpy(xlr[:,0].reshape(-1,1)).float()

#left interval limit
l=xlr[:,1]
#right interval limit
r=xlr[:,2]

ntest=len(test_x)

### feed the training data

In [ ]:
class my_dataset(Dataset):
    def __init__(self,data,label):
        self.data=x
        self.label=y          
    def __getitem__(self, index):
        return self.data[index],self.label[index]
    def __len__(self):
        return len(self.data)

### generate training data

In [ ]:
def gen3(x,q):
  z=torch.from_numpy(norm.ppf(q)).float()
  muintermediate=torch.cos(x*2*2*3.14159)
  sdintermediate=(.2+.3*(muintermediate+1))
  y=muintermediate+sdintermediate*z
  return y

experiment=3

# Collaborating Networks(gf)

In [ ]:
class cn_f(nn.Module):
    def __init__(self):
        super().__init__()
        self.k1=100
        self.k2=80
        self.k3=60
        self.fc1 = nn.Linear(2, self.k1)
        self.bn1 = nn.BatchNorm1d(self.k1)
        self.fc2 = nn.Linear(self.k1, self.k2)
        self.bn2 = nn.BatchNorm1d(self.k2)
        self.fc3 = nn.Linear(self.k2, self.k3)
        self.bn3 = nn.BatchNorm1d(self.k3)
        self.fc4 = nn.Linear(self.k3, 1)

    def forward(self, q, x):
        data=torch.cat([q,x],dim=1)
        h1 = F.elu(self.bn1(self.fc1(data)))
        h2 = F.elu(self.bn2(self.fc2(h1)))
        h3 = F.elu(self.bn3(self.fc3(h2)))
        out = self.fc4(h3)
        return out

In [ ]:
class cn_g(nn.Module):
    def __init__(self):
        super().__init__()
        self.scale=torch.sqrt(torch.from_numpy(np.array([3.29]))).float().to(device)
        self.k1=100
        self.k2=80
        self.fc1 = nn.Linear(2, self.k1)
        self.bn1 = nn.BatchNorm1d(self.k1)
        self.fc2 = nn.Linear(self.k1, self.k2)
        self.bn2 = nn.BatchNorm1d(self.k2)
        self.fc3 = nn.Linear(self.k2, 1)
        self.bn3 = nn.BatchNorm1d(1,momentum=.1,affine=False)

    def forward(self, y, x):
        data=torch.cat([y,x],dim=1)
        h1 = self.fc1(data)
        h1 = self.bn1(h1)
        h1 = F.elu(h1)
        h2 = self.fc2(h1)
        h2 = self.bn2(h2)
        h2 = F.elu(h2)
        h3 = self.fc3(h2)
        g_logit=self.bn3(h3)*self.scale
        return g_logit

In [ ]:
ns=[50,100,200,400,600,800,1000,1200,1400,1600]


In [ ]:
allll=[]
for n in ns:
    torch.manual_seed(42)
    x=torch.linspace(-.5,.5,n).reshape(-1,1)
    np.random.seed(42)
    q=np.random.rand(n,1)
    y=gen3(x,q)
    x=torch.cos(x*2*2*3.14159).reshape(-1,1)



    batch_size=200
    dataloader = DataLoader(dataset=my_dataset(x,y),
                            batch_size=batch_size,
                            shuffle=True,
                            pin_memory=True)

    pretrain_epochs =5000
    epochs = 5000
    f = cn_f().to(device)
    g = cn_g().to(device)
    floss = nn.MSELoss()
    gloss = nn.BCELoss()
    optimizer_f = optim.Adam(f.parameters(), lr=5e-4)
    optimizer_g = optim.Adam(g.parameters(), lr=1e-4)


    for epoch in notebook.trange(pretrain_epochs):
        f_running_loss = 0
        g_running_loss = 0
        for xs, ys in dataloader:
            xs, ys = xs.to(device), ys.to(device)
            optimizer_f.zero_grad()
            optimizer_g.zero_grad()
        
            q=np.random.rand(xs.shape[0],1)
            yhat=torch.rand_like(ys).to(device)*6.-2.5
  
            qhat_logit = g(yhat,xs)
            with torch.no_grad():
                ylt=ys<yhat
                ylt=ylt.float()
      
            gl = gloss(torch.sigmoid(qhat_logit),ylt)
            gl.backward(retain_graph=True)
            optimizer_g.step()

    for epoch in notebook.trange(epochs):
        f_running_loss = 0
        g_running_loss = 0
        for xs, ys in dataloader:
            xs, ys = xs.to(device), ys.to(device)
            # f update
            optimizer_f.zero_grad()
            optimizer_g.zero_grad()
            q=torch.rand(ys.shape).to(device)  
            yhat = f(q,xs)
            qhat_logit = g(yhat,xs)      
            fl = floss(q, torch.sigmoid(qhat_logit))
            fl.backward()
            optimizer_f.step()

            # g update
            optimizer_f.zero_grad()
            optimizer_g.zero_grad()
            q=torch.rand(ys.shape).to(device)  
            yhat = f(q,xs)
            qhat_logit = g(yhat,xs)
            with torch.no_grad():
                ylt=ys<yhat
                ylt=ylt.float()
            gl = gloss(torch.sigmoid(qhat_logit),ylt)
            gl.backward()
            optimizer_g.step()

            # f update
            optimizer_f.zero_grad()
            optimizer_g.zero_grad()
            q=torch.rand(ys.shape).to(device)  
            yhat = f(q,xs)
            qhat_logit = g(yhat,xs)      
            fl = floss(q, torch.sigmoid(qhat_logit))
            fl.backward()
            optimizer_f.step()

        
    f.eval()
    g.eval()

    ll_est=np.empty(ntest)


    for i in notebook.trange(ntest):
        ltmp=torch.from_numpy(np.array(l[i])).float()
        rtmp=torch.from_numpy(np.array(r[i])).float()

        if(rtmp==np.inf):
            lp=torch.sigmoid(g(ltmp.reshape(-1,1).to(device),test_x[i].reshape(-1,1).to(device)))
            lp=lp.cpu().detach()
            ll_est[i]=np.log(1.-lp+1.e-10)
        elif(ltmp==-np.inf):
            rp=torch.sigmoid(g(rtmp.reshape(-1,1).to(device),test_x[i].reshape(-1,1).to(device)))
            rp=rp.cpu().detach()
            ll_est[i]=np.log(rp+1.e-10)
        else:
            lp=torch.sigmoid(g(ltmp.reshape(-1,1).to(device),test_x[i].reshape(-1,1).to(device)))
            rp=torch.sigmoid(g(rtmp.reshape(-1,1).to(device),test_x[i].reshape(-1,1).to(device)))             
            lp=lp.cpu().detach()
            rp=rp.cpu().detach()
            ll_est[i]=np.log(rp-lp+1.e-10)
    print(np.nanmean(ll_est))
    allll.append(np.nanmean(ll_est))